# DataLab Getting Started in Python2

The Bigstep DataLab is a open data exploration service that offers data science, analytics and technology experimentation, built on our SparkArray, DataLake and on our highly flexible and high performance bare-metal infrastructure.

This tutorial assumes some programming experience.

## Uploading Data

A private datalake (HDFS service) is used to store the data that the SparkArray uses. To upload data to an HDFS cluster one would typically:
1. download the hadoop binaries (2.7.x) from a mirror like [here](http://apache.claz.org/hadoop/common/hadoop-2.7.3/hadoop-2.7.3.tar.gz)  - rather large  (240mb)
2. unarchive 
3. execute commands like "-ls".

```
/hadoop-2.7.3/bin/hdfs dfs -ls hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/
16/09/26 17:18:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
drwxrwxrwx   - hdfs supergroup          0 2016-09-22 13:12 hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/baseball
drwxrwxrwt   - hdfs supergroup          0 2016-09-22 12:08 hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/tmp
drwxr-xr-x   - hdfs supergroup          0 2016-09-22 12:08 hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/user
```

You can also execute the same commands on the master container:

In [ ]:
%%bash
wget http://www.exploredata.net/ftp/MLB2008.csv

In [ ]:
%%bash
hdfs dfs -put MLB2008.csv /

## Initialize Spark Context

For all Spark functions to be available, a Spark context has to be initialized in the current notebook.

In [ ]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate(conf=SparkConf())

## RDDs

An Resilient Distributed Dataset is an array that is spread across multiple servers. It allows the programmer to abstract away the complexity of transforming large volumes of distributed data.

In [ ]:
%%bash
wget http://seanlahman.com/files/database/baseballdatabank-master_2016-03-02.zip

apt-get install -y unzip
unzip baseballdatabank-master_2016-03-02.zip
rm -rf baseballdatabank-master_2016-03-02.zip

hdfs dfs -put baseballdatabank-master/core/AllstarFull.csv /

In [ ]:
textFile = sc.textFile("/AllstarFull.csv")

In [ ]:
textFile.count()

In [ ]:
textFile.first()

In [ ]:
linesWithRuth = textFile.filter(lambda line: "ruth" in line)

In [ ]:
linesWithRuth.count()

In [ ]:
linesWithRuth.collect()

In [ ]:
junky junky

In [ ]:
import time
linesWithRuth.saveAsTextFile("/lines_with_ruth-"+str(time.time()))

## DataFrames and SparkSQL

Using the SparkSQL module, SQL queries can be run directly on the files that are stored in Bigstep DataLake in various data formats, such as Parquet, after initializing the SparkSQL context. The type that aggregates data for SQL is the dataframe, which is built on top of RDDs.

In [ ]:
# initialize SparkSQL context

from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("demoNotebookSparkSQL").getOrCreate()
sc = spark.sparkContext

In [ ]:
vfvsgvbgbrberrb

In [ ]:
new junk here

Spark 2.1.0 has a built-in CSV reader:

In [ ]:
%%bash
hdfs dfs -chmod 740 /AllstarFull.csv

In [ ]:
allstar=spark.read.csv("/AllstarFull.csv",header=True)



In [ ]:
type(allstar)

In [ ]:
allstar.show()

In [ ]:
#register this table as a "table" within the sql context.
allstar.createOrReplaceTempView("allstar")

# SQL can be run over DataFrames that have been registered as a table.
player = spark.sql("SELECT * FROM allstar WHERE playerID like '%ruth%' and yearID<1935")

In [ ]:
player.show()

# ParquetFiles
Parquet files are typically much faster and take up less space than CSVs and the DataLake supports them as well. As Spark is a clustering system the parquet files are composed out of many fragments generated by the workres independently. The collection of files is operated as a single big table by SparkSQL.

To write the dataframe:

In [ ]:
import time
path="/allstar-"+str(time.time())+".parquet"
player.write.save(path)

Read the dataframe:

In [ ]:
dfParquet = spark.read.parquet(path)
dfParquet.createOrReplaceTempView("player")
spark.sql("SELECT playerID,YearID FROM player").show()

## External packages

There are a lot of packages that are included in the standard offering and can be seen under the 'Conda' tab in Jupyter notebook. However, we know that different use cases require a different packages in order to be able to perform advanced analytics, machine learning operations or complex visualizations. 

As a result, Bigstep SparkArray offers a simple solution to add custom packages in a specific notebook either by downloading it from the included library of installing it following the next steps:

In [ ]:
%%bash 
pip install plotly

## Advanced visualization using Plotly

Graphs help every data science team create and diseminate a powerful story built on the data they are analyzing. 

Using the default visualization libraries or external ones beautiful graphs can be created. As an example, using a standard Plotly example we can have a sneak peek into USA's flight paths from 2011:
    

In [ ]:
import pandas as pd
import plotly
from plotly.offline import iplot

plotly.offline.init_notebook_mode()

df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports.head()

df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_aa_flight_paths.csv')
df_flight_paths.head()

airports = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_airports['long'],
        lat = df_airports['lat'],
        hoverinfo = 'text',
        text = df_airports['airport'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))]
        
flight_paths = []
for i in range( len( df_flight_paths ) ):
    flight_paths.append(
        dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = [ df_flight_paths['start_lon'][i], df_flight_paths['end_lon'][i] ],
            lat = [ df_flight_paths['start_lat'][i], df_flight_paths['end_lat'][i] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            ),
            opacity = float(df_flight_paths['cnt'][i])/float(df_flight_paths['cnt'].max()),
        )
    )
    
layout = dict(
        title = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
        showlegend = False, 
        height = 800,
        geo = dict(
            scope='north america',
            projection=dict( type='azimuthal equal area' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict( data=flight_paths + airports, layout=layout )

plotly.offline.iplot(fig)
